<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


### Install spark

In [1]:
!pip install pyspark
!pip install findspark
!pip install pandas

### Import Libraries

In [2]:
import findspark
findspark.init()

In [3]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

### Importing Spark ML libraries

In [4]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

### Start session

In [5]:
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Analyse search terms on the e-commerce web server") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

23/09/16 07:33:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/16 07:33:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/16 07:33:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
# Initialize Spark session

spark

### Download The search term dataset from the below url
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [7]:
df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv')

### Load the csv into a spark dataframe

In [8]:
sdf = spark.createDataFrame(df) 

### THE TASKS BEGINS

#### Print the number of rows and columns
Take a screenshot of the code and name it as shape.jpg

In [9]:
print("Number of rows:", sdf.count())
print("Number of columns:", len(sdf.columns))

[Stage 0:>                                                          (0 + 8) / 8]

Number of rows: 10000
Number of columns: 4


#### Print the top 5 rows
Take a screenshot of the code and name it as top5rows.jpg

In [10]:
sdf.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



#### Find out the datatype of the column searchterm?
Take a screenshot of the code and name it as datatype.jpg

In [11]:
sdf.select('searchterm').dtypes[0]

('searchterm', 'string')

#### How many times was the term `gaming laptop` searched?
Take a screenshot of the code and name it as gaminglaptop.jpg)

In [12]:
# Register the DataFrame as a temporary SQL table
sdf.createOrReplaceTempView('search_data')

query1 = "SELECT COUNT(*) AS search_times FROM search_data WHERE searchterm = 'gaming laptop'"
spark.sql(query1).show()

[Stage 3:>                                                          (0 + 8) / 8]

+------------+
|search_times|
+------------+
|         499|
+------------+



#### Print the top 5 most frequently used search terms?
Take a screenshot of the code and name it as top5terms.jpg

In [13]:
query2 = """
    SELECT searchterm, COUNT(*) AS frequency
    FROM search_data
    GROUP BY searchterm
    ORDER BY frequency DESC
    LIMIT 5
"""

result = spark.sql(query2).show()

[Stage 6:===========================================>          (161 + 15) / 200]

+-------------+---------+
|   searchterm|frequency|
+-------------+---------+
|mobile 6 inch|     2312|
|    mobile 5g|     2301|
|mobile latest|     1327|
|       laptop|      935|
|  tablet wifi|      896|
+-------------+---------+



#### The pretrained sales forecasting model is available at  the below url
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [14]:
# Download File

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2023-09-16 07:34:50--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-09-16 07:34:50 (10.0 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [15]:
# Extract File
!tar -zxvf model.tar.gz

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
sales_prediction.model/data/_SUCCESS
sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
sales_prediction.model/data/._SUCCESS.crc


#### Load the sales forecast model.
Take a screenshot of the code and name it as loadmodel.jpg

In [16]:
# Using LinearRegressionModel to load the model
from pyspark.ml.regression import LinearRegressionModel

In [17]:
model = LinearRegressionModel.load('sales_prediction.model')

#### Using the sales forecast model, predict the sales for the year of 2023.
Take a screenshot of the code and name it as forecast.jpg

In [18]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','sales')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [19]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



23/09/16 07:35:26 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/09/16 07:35:26 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
